# Prerequisites

In [1]:
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import transformers

# package in . directory
from bichoice import utils
from bichoice.data_processor import (
    C3BinaryExample,
    C3BinaryDataProcessor,
)

In [2]:
assert transformers.__version__ == '4.1.1'

# declare a namespace
D = utils.GlobalSettings({
        'DATADIR': './data/',
        'MODELDIR': './outputs/bichoice_output/epoch-1/',
    })

# load training parameters
argD = utils.GlobalSettings.from_json(
    os.path.join(D.MODELDIR, 'global_settings.json'))
print('this model is trained with following hyper parameters:')
print(str(argD))

this model is trained with following hyper parameters:
{
    "MODEL_NAME": "bert-base-chinese",
    "MAX_LENGTH": 512,
    "BATCH_SIZE": 4,
    "ACCUMULATION_STEPS": 6,
    "LR": 2e-05,
    "EPOCHS": 8,
    "WARMUP_STEPS": 500,
    "MAX_GRAD_NORM": 1.0,
    "TRAIN_SET": "../outputs/csv-data/binary-train.csv",
    "VALID_SET": "../outputs/csv-data/binary-dev.csv",
    "VALID_SET_JSON": "../data/dev.json",
    "TEST_SET_JSON": "../data/test.json",
    "OUTDIR": "../outputs/bichoice_output/",
    "FP16": false,
    "DEVICE": "cuda:0"
}


# Select an Example for Test

In [3]:
def show_c3_example(e):
    '''show all info of a single example in C3'''
    print('-----PASSAGE-----')
    print('\n'.join(e.sentences))
    print('-----QUESTION-----')
    print(e.question)
    print('-----OPTIONS-----')
    for i, o in enumerate(e.options):
        print('    {}: {}'.format(chr(i+ord('A')), o))
    print('-----ANSWER-----')
    l = e.label
    print('    {}: {}'.format(chr(l+ord('A')), e.options[l]))

In [4]:
test = utils.get_all_C3examples(os.path.join(D.DATADIR, 'test.json'))
test_e = random.choice(test)
show_c3_example(test_e)

-----PASSAGE-----
有一个博物馆被盗了，丢失了十件珍贵的文物，幸好一枚珍贵的钻石戒指没有被盗。警方经过多次努力也找不到线索，这时，一直很冷静的博物馆馆长提议让电视台采访他。不久，电视上播放了记者采访博物馆馆长的镜头。记者问：“这次共丢失了多少件文物?”馆长答：“十一件。”记者又问：“这些文物都很珍贵吗?”馆长答：“是的，都很珍贵，特别是一枚钻戒，价值连城。”时隔不久，警方就查到了线索，顺利地破了案。线索来源很简单，几个盗贼在互相殴打时被警方抓获，而他们殴斗的原因竟然是互相猜疑究竟是谁私藏了第十一件文物——那枚珍贵的钻戒。
-----QUESTION-----
博物馆被盗后，馆长做了什么?
-----OPTIONS-----
    A: 亲自去参加调查
    B: 请记者来采访他
    C: 转移走那枚钻戒
    D: 看电视采访录像
-----ANSWER-----
    B: 请记者来采访他


# Reshape C3 as Binary Classification

In [5]:
def C3Example_to_C3BinaryExample(eList):
    '''
    create `C3BinaryExample`s from `C3Example`s.
    
    Args
    ----
    `eList` : a list of `bichoice.utils.C3Example`
    
    Return
    ------
    `out` : a list of `bichioce.data_processor.C3BinaryExample`.
    '''
    out = []
    for e in eList:
        passage = ''.join(e.sentences)
        question = e.question
        answer = e.options[e.label]
        for o in e.options:
            if o == answer:
                continue
            out.append(C3BinaryExample(passage, question, answer, o, 0))
            out.append(C3BinaryExample(passage, question, o, answer, 1))
    return out

In [6]:
def show_c3_binary_example(e):
    '''show all info of a single `C3BinaryExample`'''
    print('-----PASSAGE-----')
    print(e.passage)
    print('-----QUESTION-----')
    print(e.question)
    print('-----CHOICE_0-----')
    print(e.choice_0)
    print('-----CHOICE_1-----')
    print(e.choice_1)
    print('-----LABEL-----')
    print(e.label)

In [7]:
# decomposite this example to several binary examples
test_b_e = C3Example_to_C3BinaryExample([test_e])
for i, e in enumerate(test_b_e):
    print('-----EXAMPLE{}-----'.format(i+1))
    show_c3_binary_example(e)

-----EXAMPLE1-----
-----PASSAGE-----
有一个博物馆被盗了，丢失了十件珍贵的文物，幸好一枚珍贵的钻石戒指没有被盗。警方经过多次努力也找不到线索，这时，一直很冷静的博物馆馆长提议让电视台采访他。不久，电视上播放了记者采访博物馆馆长的镜头。记者问：“这次共丢失了多少件文物?”馆长答：“十一件。”记者又问：“这些文物都很珍贵吗?”馆长答：“是的，都很珍贵，特别是一枚钻戒，价值连城。”时隔不久，警方就查到了线索，顺利地破了案。线索来源很简单，几个盗贼在互相殴打时被警方抓获，而他们殴斗的原因竟然是互相猜疑究竟是谁私藏了第十一件文物——那枚珍贵的钻戒。
-----QUESTION-----
博物馆被盗后，馆长做了什么?
-----CHOICE_0-----
请记者来采访他
-----CHOICE_1-----
亲自去参加调查
-----LABEL-----
0
-----EXAMPLE2-----
-----PASSAGE-----
有一个博物馆被盗了，丢失了十件珍贵的文物，幸好一枚珍贵的钻石戒指没有被盗。警方经过多次努力也找不到线索，这时，一直很冷静的博物馆馆长提议让电视台采访他。不久，电视上播放了记者采访博物馆馆长的镜头。记者问：“这次共丢失了多少件文物?”馆长答：“十一件。”记者又问：“这些文物都很珍贵吗?”馆长答：“是的，都很珍贵，特别是一枚钻戒，价值连城。”时隔不久，警方就查到了线索，顺利地破了案。线索来源很简单，几个盗贼在互相殴打时被警方抓获，而他们殴斗的原因竟然是互相猜疑究竟是谁私藏了第十一件文物——那枚珍贵的钻戒。
-----QUESTION-----
博物馆被盗后，馆长做了什么?
-----CHOICE_0-----
亲自去参加调查
-----CHOICE_1-----
请记者来采访他
-----LABEL-----
1
-----EXAMPLE3-----
-----PASSAGE-----
有一个博物馆被盗了，丢失了十件珍贵的文物，幸好一枚珍贵的钻石戒指没有被盗。警方经过多次努力也找不到线索，这时，一直很冷静的博物馆馆长提议让电视台采访他。不久，电视上播放了记者采访博物馆馆长的镜头。记者问：“这次共丢失了多少件文物?”馆长答：“十一件。”记者又问：“这些文物都很珍贵吗?”馆长答：“是的，都很珍贵，特别是一枚钻戒，价值连城。

# Infer with Bi-Chioce Model

In [8]:
# initialize model and load state dict from a checkpoint 
D.DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = transformers.BertTokenizer.from_pretrained(argD.MODEL_NAME)
processor = C3BinaryDataProcessor(tokenizer, argD.MAX_LENGTH)
model = transformers.BertForSequenceClassification.from_pretrained(
    argD.MODEL_NAME, num_labels=2)
model.load_state_dict(torch.load(os.path.join(D.MODELDIR, 'model.bin')))
model.to(D.DEVICE)
model.eval()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
def compare(D, model, processor, passage, question, c1, c2, avg=True):
    '''
    compare two options and select one with the higher probability.

    Args
    ----
    `D` : instance of `GlobalSettings`
    
    `model` : binary classifier

    `processor` : instance of `C3BinaryDataProcessor`

    `passage` : str

    `question` : str

    `c1` : str

    `c2` : str

    `avg` : bool, if `True`, the we switch the position of `c1` and `c2` 
            and average output as probability. 

    Return
    ------
    `label` : int, 0 means selecting c1, 1 means selecting c2
    '''
    # tokenizing
    example1 = C3BinaryExample(passage, question, c1, c2)
    example2 = C3BinaryExample(passage, question, c2, c1)
    f1 = processor.convert_example_to_features(example1)
    f2 = processor.convert_example_to_features(example2)
    batch = {
'input_ids': torch.LongTensor([f1.input_ids, f2.input_ids]),
'attention_mask': torch.LongTensor([f1.input_mask,f2.input_mask]),
'token_type_ids': torch.LongTensor([f1.segment_ids,f2.segment_ids]),
    }
    for k in batch:
        batch[k] = batch[k].to(D.DEVICE)

    model.to(D.DEVICE)
    model.eval()
    with torch.no_grad():
        output = model(**batch)
        logits = output.logits
    if avg:
        logit1 = (logits[0,0] + logits[1,1]).item()
        logit2 = (logits[0,1] + logits[1,0]).item()
    else:
        logit1 = logits[0,0].item()
        logit2 = logits[0,1].item()

    return int(logit2 > logit1)

In [ ]:
candidate = test_e.options[0]
# we sequentially compare all options
for c2 in test_e.options[1:]:
    print('we compare following options:')
    print('    option1:', candidate)
    print('    option2:', c2)
    bin_label = compare(D, model, processor, ''.join(test_e.sentences), 
                        test_e.question, candidate, c2)
    if bin_label == 1:
        print('and we select:', c2)
        candidate = c2
    else:
        print('and we select:', candidate)
print('----------')
print('infered answer:', candidate)
print('correct answer:', test_e.options[test_e.label])